In [2]:
%cd Exploiting-Food-Embeddings-for-Ingredient-Substitution/

[WinError 2] The system cannot find the file specified: 'Exploiting-Food-Embeddings-for-Ingredient-Substitution/'
c:\Users\hp\Embeddings Project\Exploiting-Food-Embeddings-for-Ingredient-Substitution


In [3]:
# For Linux
# !wget https://github.com/ChantalMP/Exploiting-Food-Embeddings-for-Ingredient-Substitution/releases/download/0.1/food2vec_models.zip

# For Windows
!Invoke-WebRequest -Uri "https://github.com/ChantalMP/Exploiting-Food-Embeddings-for-Ingredient-Substitution/releases/download/0.1/food2vec_models.zip" -OutFile "food2vec_models.zip"

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
# For Linux
# !unzip -qq food2vec_models.zip

# For Windows
!Expand-Archive -Path "food2vec_models.zip" -DestinationPath "food2vec_models"


# For both
!mv food2vec_models/* food2vec/models/
!rm -r food2vec_models

'unzip' is not recognized as an internal or external command,
operable program or batch file.
'mv' is not recognized as an internal or external command,
operable program or batch file.
'rm' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
!pip install -r requirements.txt

In [6]:
from gensim.models import Word2Vec
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import openai
import pandas as pd
import openai

In [7]:
with open('foodbert/data/used_ingredients.json', 'r') as f:
    used_ingredients = json.load(f)

In [8]:
data = pd.read_csv("C:/Users/hp\Downloads/Ingredients Dinnerfy Airtable.csv")

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2038 entries, 0 to 2037
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Name     2037 non-null   object
 1   Recipes  1122 non-null   object
 2   look     2038 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 47.9+ KB


In [10]:
my_ingredients = list(data["Name"].astype("str"))
# print(my_ingredients)
len(my_ingredients)

2038

In [11]:
map_obj = map(lambda x: x.lower(), my_ingredients)

In [12]:
my_ingredients = list(map_obj)
# print(my_ingredients)
len(my_ingredients)

2038

In [13]:
# Loading the food2vec model 
model = Word2Vec.load('food2vec/models/food2vec_models/model.bin')

FileNotFoundError: [Errno 2] No such file or directory: 'food2vec/models/food2vec_models/model.bin'

In [ ]:
# Function for getting the embeddings of ingredients from OpenAI Embeddings Model
def get_embeddings(input):
    response = openai.Embedding.create(
        input= input,
        model="text-embedding-ada-002"
    )
    embeddings = response['data'][0]['embedding']

    return np.array(embeddings)

In [ ]:
# Function to evaluate the similarity of two embeddings

def similarity(w1, w2):
    similarity = cosine_similarity(w1.reshape(1, -1), w2.reshape(1, -1))
    return f"Cosine Similarity: {similarity[0][0]}"

### Evaluation OpenAI's Embeddings of Ingredients

In [ ]:
salt = get_embeddings("salt")
kosher_salt = get_embeddings("kosher_salt")
pepper= get_embeddings("pepper")

In [ ]:
# OpenAI makes embeddings of dimension 1536
len(salt)

1536

In [ ]:
print(f"Salt and Pepper - {similarity(salt, pepper)}")
print(f"Salt and Kosher Salt - {similarity(salt, kosher_salt)}")

Salt and Pepper - Cosine Similarity: 0.829462213227012
Salt and Kosher Salt - Cosine Similarity: 0.854892319267522


Salt and Kosher Salt have a high cosine similarity but salt and pepper also have a reasonably high cosine similarity and their is only a marginal difference between the two cosine similarities.

In [ ]:
parmesan  = get_embeddings("parmesan_cheese")
grated_parmesan  = get_embeddings("grated_parmesan_cheese")
mozzarella = get_embeddings("mozzarella_cheese")

In [ ]:
print(f"Parmesan Cheese and Grated Parmesan Cheese - {similarity(parmesan, grated_parmesan)}")
print(f"Parmesan Cheese and Mozzarella Cheese - {similarity(parmesan, mozzarella)}")

Parmesan Cheese and Grated Parmesan Cheese - Cosine Similarity: 0.9385673402474369
Parmesan Cheese and Mozzarella Cheese - Cosine Similarity: 0.9329625437248483


Parmesan Cheese's cosine similarity with grated parmesan cheese is almost equal to its cosine similarity with mozzarella cheese, however, the cosine similarity with grated parmesan cheese should have been more since its just a different form of the same ingredient.

In [ ]:
garlic  = get_embeddings("garlic")
garlic_clove  = get_embeddings("garlic_clove")
ginger = get_embeddings("ginger")

In [ ]:
print(f"Garlic and Garlic Cloves - {similarity(garlic, garlic_clove)}")
print(f"Garlic and Ginger - {similarity(garlic, ginger)}")

Garlic and Garlic Cloves - Cosine Similarity: 0.9181661152944927
Garlic and Ginger - Cosine Similarity: 0.8690165790494919


### Observation on OpenAI Embeddings

The embeddings are representative of the strong similarity between any two similar ingredients. However, they tend to have a high cosine similarity even between ingredients which are not similar, which makes it difficult to identify the most similar ingredients and different names of the same ingredient.

### Evaluation of Food2Vec Embeddings

In [ ]:
# Length of Food2Vec Embeddings is 100
len(model.wv["salt"])

100

In [ ]:
w1 = ["kosher"]
model.wv.most_similar (positive=w1,topn=6)

[('Kosher', 0.6294300556182861),
 ('cracked_black_pepper', 0.6071157455444336),
 ('Fresh-', 0.5951094031333923),
 ('dried_parsley', 0.5932490229606628),
 ('maldon', 0.5932028293609619),
 ('morton', 0.5621879696846008)]

In [ ]:
print(f"Cosine Similarity between salt and kosher_salt - {model.wv.similarity(w1='salt', w2 ='kosher_salt')}")
print(f"Cosine Similarity between salt and pepper - {model.wv.similarity(w1='salt', w2 ='pepper')}")

Cosine Similarity between salt and kosher_salt - 0.8143872618675232
Cosine Similarity between salt and pepper - 0.18186870217323303


In [ ]:
print(f"Cosine Similarity between parmesan_cheese and grated_parmesan_cheese - {model.wv.similarity(w1='parmesan_cheese', w2 ='grated_parmesan_cheese')}")
print(f"Cosine Similarity between parmesan_cheese and mozzarella_cheese - {model.wv.similarity(w1='parmesan_cheese', w2 ='mozzarella_cheese')}")

Cosine Similarity between parmesan_cheese and grated_parmesan_cheese - 0.7766836285591125
Cosine Similarity between parmesan_cheese and mozzarella_cheese - 0.8087217211723328


In [ ]:
model.wv.most_similar("parmesan_cheese", topn = 5)

[('parmesan', 0.9504023790359497),
 ('romano_cheese', 0.8272979259490967),
 ('mozzarella_cheese', 0.8087217211723328),
 ('cheddar_cheese', 0.7970229983329773),
 ('parmigiano', 0.780430257320404)]

In [ ]:
print(f"Cosine Similarity between garlic and garlic_clove - {model.wv.similarity(w1='garlic', w2 ='garlic_clove')}")
print(f"Cosine Similarity between garlic and ginger - {model.wv.similarity(w1='garlic', w2 ='ginger')}")

Cosine Similarity between garlic and garlic_clove - 0.6446242332458496
Cosine Similarity between garlic and ginger - 0.5138317346572876


In [ ]:
model.wv.most_similar("garlic", topn = 5)

[('minced_garlic', 0.8553013205528259),
 ('crushed_garlic', 0.8153051137924194),
 ('chopped_garlic', 0.7908270955085754),
 ('garlic_paste', 0.7144184708595276),
 ('garlic;', 0.697868287563324)]

### Evaluation of FoodBERT 

In [ ]:
!wget https://github.com/ChantalMP/Exploiting-Food-Embeddings-for-Ingredient-Substitution/releases/download/0.1/foodbert_data.zip

# For Linux
# !wget https://github.com/ChantalMP/Exploiting-Food-Embeddings-for-Ingredient-Substitution/releases/download/0.1/foodbert_data.zip

# For Windows
!Invoke-WebRequest -Uri "https://github.com/ChantalMP/Exploiting-Food-Embeddings-for-Ingredient-Substitution/releases/download/0.1/foodbert_data.zip" -OutFile "food_models.zip"

In [ ]:
!unzip -qq foodbert_data.zip
!mv foodbert_data/* foodbert/data/
!rm -r foodbert_data

In [ ]:
!pip3 install torch torchvision torchaudio

In [2]:
from transformers import BertModel, BertForMaskedLM, BertTokenizer
import torch
import json

c:\Users\hp\Embeddings Project\Exploiting-Food-Embeddings-for-Ingredient-Substitution\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [89]:
with open('foodbert/data/used_ingredients.json', 'r') as f:
    used_ingredients = json.load(f)
tokenizer = BertTokenizer(vocab_file='foodbert/data/bert-base-cased-vocab.txt', do_lower_case=False, max_len=128, never_split=used_ingredients)

In [93]:
model = BertModel.from_pretrained(pretrained_model_name_or_path='foodbert/data/foodbert_data/mlm_output/checkpoint-final')
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(33247, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [303]:
def get_bert_embedding(word):
    """
    Returns the average of the token indices except the last (EOS) and the first token (BOS)
    """

    input_ids = torch.tensor(tokenizer.encode(word, add_special_tokens=True)).unsqueeze(0)
    input_ids = input_ids.to(device)
    outputs = model(input_ids)
    embedding = np.sum(outputs[0].detach().numpy()[:,0:-1,:],axis = 1)/(outputs[0].detach().numpy().shape[1] - 2)

    return embedding


In [304]:
# FoodBERT generates 768 dimensional embeddings for ingredients

get_bert_embedding("salt").shape


(1, 768)

In [246]:
salt = get_bert_embedding("salt")
kosher_salt = get_bert_embedding("kosher_salt")
pepper = get_bert_embedding("pepper")

print(f"Salt and Kosher Salt - {similarity(salt, kosher_salt)}")
print(f"Salt and Pepper - {similarity(salt, pepper)}")

Salt and Kosher Salt - Cosine Similarity: 0.8881312608718872
Salt and Pepper - Cosine Similarity: 0.6598259806632996


FoodBERT is clearly able to highlight the strong similarity between salt and kosher salt and also the low similarity between salt and pepper.

In [310]:
parmesan = get_bert_embedding("parmesan_cheese")
grated_parmesan = get_bert_embedding("grated_parmesan_cheese")
mozzarella = get_bert_embedding("mozzarella_cheese")

print(f"Parmesan Cheese and Grated Parmesan Cheese - {similarity(parmesan, grated_parmesan)}")
print(f"Parmesan Cheese and Mozzarella Cheese - {similarity(parmesan, mozzarella)}")

Parmesan Cheese and Grated Parmesan Cheese - Cosine Similarity: 0.8406903743743896
Parmesan Cheese and Mozzarella Cheese - Cosine Similarity: 0.8513528108596802


Embeddings for Parmesan Cheese and Grated Parmesan Cheese and parmesan cheese and mozzarella cheese have a high cosine similarity.

In [248]:
garlic = get_bert_embedding("garlic")
garlic_clove = get_bert_embedding("garlic_clove")
ginger = get_bert_embedding("ginger")

print(f"Garlic and Garlic Cloves - {similarity(garlic, garlic_clove)}")
print(f"Garlic and Ginger - {similarity(garlic, ginger)}")

Garlic and Garlic Cloves - Cosine Similarity: 0.8596292734146118
Garlic and Ginger - Cosine Similarity: 0.5114772915840149


Garlic and Garlic Cloves have a high cosine similarity which is valid, since they are synonymous, while the cosine similarity for Garlic and Ginger is low which is also valid.

In [251]:
potato = get_bert_embedding("potato")
sweet_potato = get_bert_embedding("sweet_potato")
sliced_potato = get_bert_embedding("sliced_potato")

print(f"Potato and Sliced Potato - {similarity(potato, sliced_potato)}")
print(f"Potato and Sweet Potato - {similarity(potato, sweet_potato)}")

Potato and Sliced Potato - Cosine Similarity: 0.8131145238876343
Potato and Sweet Potato - Cosine Similarity: 0.7819738984107971


Cosine Similarity between Potato and Sliced Potato is higher than the cosine similarity between Potato and Sweet Potato which is correct, since sliced potato is processed form of potatoes while sweet potato is a different ingredient.

In [297]:
Crushed_Red_Pepper_Flakes = get_bert_embedding("crushed_red_pepper_flakes")
Chilli_Flakes = get_bert_embedding("chilli_flakes")
Oregano_Seasoning = get_bert_embedding("oregano_seasoning")

print(f"Crushed_Red_Pepper_Flakes and Chilli Flakes - {similarity(Crushed_Red_Pepper_Flakes, Chilli_Flakes)}")
print(f"Crushed_Red_Pepper_Flakes and Oregano Seasoning - {similarity(Crushed_Red_Pepper_Flakes, Oregano_Seasoning)}")

Crushed_Red_Pepper_Flakes and Chilli Flakes - Cosine Similarity: 0.8620383739471436
Crushed_Red_Pepper_Flakes and Oregano Seasoning - Cosine Similarity: 0.7542827129364014


FoodBERT is able to generate apt embeddings for new ingredients like crushed_red_pepper flakes and also obtain the right relationship between new ingredients.

In [282]:
onion = get_bert_embedding("onion")
large_onion = get_bert_embedding("large_onion")

print(f"Onion and Large Onion - {similarity(onion, large_onion)}")

Onion and Large Onion - Cosine Similarity: 0.7472556233406067


Onion and Large Onion are the same ingredients, however, they have a relatively low degree of similarity

In [259]:
egg = get_bert_embedding("egg")
boiled_egg = get_bert_embedding("boiled_egg")
eggplant = get_bert_embedding("eggpant")

print(f"Egg and Boiled Egg - {similarity(egg, boiled_egg)}")
print(f"Egg and Eggplant - {similarity(egg, eggplant)}")

Egg and Boiled Egg - Cosine Similarity: 0.6770237684249878
Egg and Eggplant - Cosine Similarity: 0.7074657678604126


Egg and boiled egg are the same ingredients, however, their cosine similarity is low. On the other hand, egg and eggplant have a higher cosine similarity which are completely different food items.

In [285]:
potato = get_bert_embedding("potato")
potatoes = get_bert_embedding("potatoes")


print(f"Potato and Potatoes - {similarity(potato, potatoes)}")

Potato and Potatoes - Cosine Similarity: 0.6225559115409851


Potato and potatoes have a very low cosine similarity even though they are the same ingredients

In [286]:
tomato = get_bert_embedding("tomato")
cherry_tomato = get_bert_embedding("cherry_tomato")

print(f"Tomato and Cherry Tomato - {similarity(tomato, cherry_tomato)}")

Tomato and Cherry Tomato - Cosine Similarity: 0.7752974629402161


### Observations on FoodBERT Embeddings

#### Advantages:
- Able to generate reasonable embeddings for previously unseen ingredients 
- Performs adequately in differentiating between closely used ingredients (like salt and pepper)

#### Limitations:
- Performs poorly on plural form of the same ingredient
- All ingredients should be in the same form (like "FirstName_LastName") otherwise can get unexpected results
- Poor results with prefixes like "large" before the ingredient


#### Ways to Use
- Embeddings for a good database of ingredients can be generated and stored in a database and the unknown ingredients can be matched with the already existing database by calculating cosine similarity
- Prefixes like "large" can be removed from the ingredients can be removed 